## Set Up

In [10]:
# Install the required packages
!pip install langchain-groq langchain_community langchain-core pydantic bs4


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import getpass
import os
import requests

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

GROQ_API_KEY:  ········


In [4]:
_set_env("LANGSMITH_API_KEY")

LANGSMITH_API_KEY:  ········


## LLM SET UP

In [5]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Load and process the LCEL documentation
url = "https://python.langchain.com/docs/concepts/lcel/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import os

# Define the prompt template
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user question based on the \n 
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    </instructions> \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

# Initialize the Groq client
api_key = os.environ.get("GROQ_API_KEY")
expt_llm = "llama-3.1-8b-instant"
llm = ChatGroq(temperature=0, model_name=expt_llm, api_key=api_key)

# Data model for the code solution
class Code(BaseModel):
    """Schema for code solutions to questions about LCEL."""
    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")

# Add structured output support
structured_llm = llm.with_structured_output(Code, include_raw=True)

# Function to check for errors in the output
def check_groq_output(tool_output):
    """Check for parse error or failure to call the tool."""
    # Error with parsing
    if tool_output.get("parsing_error"):
        print("Parsing error!")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )

    # Tool was not invoked
    elif not tool_output.get("parsed"):
        print("Failed to invoke tool!")
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    
    return tool_output

# Chain with output check
code_chain_groq = code_gen_prompt | structured_llm | check_groq_output

# Function to parse the output into the Code schema
def parse_output(response: str) -> Code:
    """Parse the response into the Code schema."""
    try:
        # Attempt to split the response into prefix, imports, and code
        if "Imports:" in response and "Code:" in response:
            prefix = response.split("Imports:")[0].strip()
            imports = response.split("Imports:")[1].split("Code:")[0].strip()
            code = response.split("Code:")[1].strip()
        else:
            # If the response doesn't contain the expected delimiters, use the entire response as the code
            prefix = ""
            imports = "No imports provided."
            code = response.strip()
        
        return Code(prefix=prefix, imports=imports, code=code)
    except Exception as e:
        raise ValueError(f"Error parsing response: {e}")

# Optional: With re-try to correct for failure to invoke tool
def insert_errors(inputs):
    """Insert errors for tool parsing in the messages."""
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        HumanMessage(
            content=f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool."
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }

# Fallback chain for retries
fallback_chain = insert_errors | code_chain_groq
N = 3  # Max re-tries
code_gen_chain_with_retry = code_chain_groq.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)

# Final chain with retry and output parsing
code_gen_chain = code_gen_chain_with_retry | parse_output

from tenacity import retry, stop_after_attempt, wait_exponential
import time

# Function to handle rate limits
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=10))
def safe_invoke(chain, input_data):
    try:
        return chain.invoke(input_data)
    except groq.RateLimitError as e:
        print(f"Rate limit exceeded. Retrying in {e.retry_after} seconds...")
        time.sleep(e.retry_after)
        raise


# Example usage
question = "How do I build a RAG chain in LCEL?"
try:
    # Invoke the chain
    response = safe_invoke(code_gen_chain, {"context": concatenated_content, "messages": [("user", question)]})
    
    # Print the solution
    print("Prefix:", response.prefix)
    print("Imports:", response.imports)
    print("Code:", response.code)
except Exception as e:
    print(f"Error: {e}")

Error: RetryError[<Future at 0x1a16a750d30 state=finished raised AttributeError>]


In [40]:
response

Code(prefix='Building a RAG chain in LCEL', imports='from langchain.chains import RetrievalAugmentedGenerator, LLMAgent, WebSearch, Chain', code="def build_rag_chain():\n    # Define the components of the RAG chain\n    llm = LLMAgent(model_name='text-davinci-003', temperature=0.7)\n    retriever = WebSearch()\n    generator = RetrievalAugmentedGenerator(llm, retriever)\n\n    # Define the RAG chain\n    chain = Chain([generator])\n\n    return chain")

## State

In [15]:
from typing import List
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: str
    messages: List
    generation: str
    iterations: int

## Graph

In [16]:
### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes


def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [17]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [18]:
question = "How can I directly pass a string to a runnable and use it to construct the input needed for my prompt?"
solution = app.invoke({"messages": [("user", question)], "iterations": 0, "error": ""})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---


<string>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.


---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: FINISH---


In [19]:
solution["generation"]

Code(prefix='langchain_core.runnables', imports='from langchain.chains import LLMChain', code='def construct_input(input_string):\n    prompt_template = "Given the string " + input_string + ", the answer is " + input_string\n    return LLMChain(llm_name="text-davinci-003", prompt=prompt_template)\n\n# Invoke the runnable with a string input\nresult = construct_input("Hello, World!").invoke()\n')

In [20]:
import langsmith

client = langsmith.Client(api_key = os.getenv("LANGSMITH_API_KEY"))

In [21]:
try:
    public_dataset = (
        "https://smith.langchain.com/public/326674a6-62bd-462d-88ae-eea49d503f9d/d"
    )
    client.clone_public_dataset(public_dataset)
except Exception as e:
    print("Error cloning dataset:", str(e))


In [22]:
datasets = client.list_datasets()
for dataset in datasets:
    print(dataset)


name='lcel-teacher-eval' description='Eval set for LCEL teacher' data_type=<DataType.kv: 'kv'> id=UUID('5451bb2d-5a52-41a7-aa53-5b12ea1d7f02') created_at=datetime.datetime(2025, 3, 2, 16, 37, 57, 273231, tzinfo=datetime.timezone.utc) modified_at=datetime.datetime(2025, 3, 2, 16, 37, 57, 273231, tzinfo=datetime.timezone.utc) example_count=20 session_count=5 last_session_start_time=datetime.datetime(2025, 3, 2, 17, 56, 31, 926449) inputs_schema={'type': 'object', 'title': 'dataset_input_schema', 'required': ['question'], 'properties': {'question': {'type': 'string'}}} outputs_schema={'type': 'object', 'title': 'dataset_output_schema', 'required': ['answer'], 'properties': {'answer': {'type': 'string'}}} transformations=None


In [23]:
from langsmith.schemas import Example, Run


def check_import(run: Run, example: Example) -> dict:
    imports = run.outputs.get("imports")
    try:
        exec(imports)
        return {"key": "import_check", "score": 1}
    except Exception:
        return {"key": "import_check", "score": 0}


def check_execution(run: Run, example: Example) -> dict:
    imports = run.outputs.get("imports")
    code = run.outputs.get("code")
    try:
        exec(imports + "\n" + code)
        return {"key": "code_execution_check", "score": 1}
    except Exception:
        return {"key": "code_execution_check", "score": 0}

In [27]:
def predict_base_case(example: dict):
    """Context stuffing"""
    solution = safe_invoke(code_gen_chain, 
        {"context": concatenated_content, "messages": [("user", example["question"])]}
    )
    return {"imports": solution.imports, "code": solution.code}


def predict_langgraph(example: dict):
    """LangGraph"""
    graph = safe_invoke(app,
        {"messages": [("user", example["question"])], "iterations": 0, "error": ""}
    )
    solution = graph["generation"]
    return {"imports": solution.imports, "code": solution.code}

In [25]:
from langsmith.evaluation import evaluate

# Evaluator
code_evalulator = [check_import, check_execution]

# Dataset
dataset_name = "lcel-teacher-eval"

In [26]:
# Run base case
try:
    experiment_results_ = client.evaluate(
        predict_base_case,
        data=dataset_name,
        evaluators=code_evalulator,
        experiment_prefix=f"test-without-langgraph-{expt_llm}",
        max_concurrency=2,
        metadata={
            "llm": expt_llm,
        },
    )
except Exception as e:
    print(f"LangSmith setup error: {e}")

View the evaluation results for experiment: 'test-without-langgraph-llama-3.1-8b-instant-35ed37fb' at:
https://smith.langchain.com/o/2dbf5add-f9e9-4edb-b995-076e3f219725/datasets/5451bb2d-5a52-41a7-aa53-5b12ea1d7f02/compare?selectedSessions=753a04b7-79ee-4311-9020-25284ec2905c




0it [00:00, ?it/s]

Error running target function: RetryError[<Future at 0x1a166a2a790 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3729, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
  File "D:\Conda\lib\site-packag

Parsing error!
Parsing error!
Parsing error!
Parsing error!
Parsing error!
Parsing error!
Parsing error!


Error running target function: RetryError[<Future at 0x1a16abfea90 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 4721, in invoke
    return self._call_with_config(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", li

Parsing error!
Parsing error!


Error running target function: RetryError[<Future at 0x1a16a201b50 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 4721, in invoke
    return self._call_with_config(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", li

Parsing error!
Parsing error!


In [28]:
try:
    experiment_results_ = client.evaluate(
        predict_base_case,
        data=dataset,
        evaluators=code_evalulator,
        experiment_prefix=f"test-without-langgraph-{expt_llm}",
        max_concurrency=2,
        metadata={"llm": expt_llm},
    )
except Exception as e:
    print(f"LangSmith setup error: {e}")


View the evaluation results for experiment: 'test-without-langgraph-llama-3.1-8b-instant-870d45a7' at:
https://smith.langchain.com/o/2dbf5add-f9e9-4edb-b995-076e3f219725/datasets/5451bb2d-5a52-41a7-aa53-5b12ea1d7f02/compare?selectedSessions=a133b5cd-7463-483d-86f5-fbfe03a8b19c




0it [00:00, ?it/s]

Parsing error!
Parsing error!
Parsing error!
Parsing error!


Error running target function: RetryError[<Future at 0x1a16a0c8dc0 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 4721, in invoke
    return self._call_with_config(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", li

Parsing error!


Error running target function: RetryError[<Future at 0x1a16a1be2b0 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3729, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
  File "D:\Conda\lib\site-packag

Parsing error!
Parsing error!


Error running target function: RetryError[<Future at 0x1a16a298940 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 4721, in invoke
    return self._call_with_config(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", li

Parsing error!
Parsing error!
Parsing error!
Parsing error!


Error running target function: RetryError[<Future at 0x1a16a667c10 state=finished raised NameError>]
Traceback (most recent call last):
  File "C:\Users\kuany\AppData\Local\Temp\ipykernel_11564\4984125.py", line 101, in safe_invoke
    return chain.invoke(input_data)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 199, in invoke
    raise first_error
  File "D:\Conda\lib\site-packages\langchain_core\runnables\fallbacks.py", line 179, in invoke
    output = context.run(
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3024, in invoke
    input = context.run(step.invoke, input, config)
  File "D:\Conda\lib\site-packages\langchain_core\runnables\base.py", line 3729, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
  File "D:\Conda\lib\site-packag